# Capillary Rise - benchmark test case (SFB 1194)

Results are published in *A comparative study of transient capillary rise using direct numerical simulations* (https://linkinghub.elsevier.com/retrieve/pii/S0307904X20302134)

In [ ]:
//#r "BoSSSpad/BoSSSpad.dll"
#r "../../src/L4-application/BoSSSpad/bin/Release/net6.0/BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.LevelSetTools.SolverWithLevelSetUpdater;
using BoSSS.Solution.NSECommon;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [ ]:
var myBatch = ExecutionQueues[1];
myBatch

RuntimeLocation,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,Username,ServerName,ComputeNodes,DefaultJobPriority,SingleNode,AllowedDatabasesPaths
win\amd64,\\hpccluster\hpccluster-scratch\smuda\binaries,True,FDY-WindowsHPC,dotnet,FDY\smuda,DC2,<null>,Normal,True,[ \\hpccluster\hpccluster-scratch\smuda\ ]


In [ ]:
BoSSSshell.WorkflowMgm.Init("CapillaryRise", myBatch);

Project name is set to 'CapillaryRise'.
Opening existing database '\\hpccluster\hpccluster-scratch\smuda\CapillaryRise'.


In [ ]:
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();

## Test case setup (physical parameters)

In [ ]:
bool startUp = true;

In [ ]:
string omegaTc = "Omega1";     // {"Omega0.1", "Omega0.5", "Omega1", "Omega10", "Omega100"}

In [ ]:
double R = 5.0e-3;      // in cm
double H = 4.0e-2;

double rhoA = 0.0;
double muA = 0.01;
double rhoB = 0.0;
double muB = 0.01 / 1000;
double sigma = 0.0;

double Lslip = R / 5.0;
double betaS_A = (muA / Lslip);
double betaS_B = (muB / Lslip);

double betaL = 0;
double theta_e = 3.0 * Math.PI / 18.0;

double g = 0.0;
Formula GravityValue;

double t_end = 0.0;
double t_startUp = 0.0;

double dt = 0.0;
double dt_startUp = 0.0;

Guid restartID = new Guid();
int ts_restart = 0;

In [ ]:
switch(omegaTc) {
    case "Omega0.1": {

            rhoA = 1663.8;
            rhoB = 1663.8 / 1000;      
            sigma = 0.2;       // kg / s^2

            g = 1.04;         // cm / s^2
            GravityValue = new Formula(
                "GravY",
                false,
                "double GravY(double[] X) { return -1.04; } "
            );

            t_end = 13.86;
            t_startUp = 0.0; 

            dt = 2e-5; 
            dt_startUp = 3e-5;

            //restartID = new Guid("");   // startUp session

            break;
        }
    case "Omega0.5": {

            rhoA = 133.0;
            rhoB = 133.0 / 1000;      
            sigma = 0.1;       // kg / s^2

            g = 6.51;         // cm / s^2
            GravityValue = new Formula(
                "GravY",
                false,
                "double GravY(double[] X) { return -6.51; } "
            );

            t_end = 1.11;
            t_startUp = 0.0;

            dt = 2e-5;
            dt_startUp = 4e-5;

            //restartID = new Guid("");   // startUp session

            break;
        }
    case "Omega1": {

            rhoA = 83.1;
            rhoB = 83.1 / 1000;      
            sigma = 0.04;       // kg / s^2

            g = 4.17;         // cm / s^2
            GravityValue = new Formula(
                "GravY",
                false,
                "double GravY(double[] X) { return -4.17; } "
            );

            t_end = 0.7;
            t_startUp = 0.0;

            dt = 1e-5;
            dt_startUp = 5e-4;

            //restartID = new Guid("");   // startUp session

            break;
        }
    case "Omega10": {

            rhoA = 3.3255;
            rhoB = 3.3255 / 1000;      
            sigma = 0.01;       // kg / s^2

            g = 26.042;         // cm / s^2
            GravityValue = new Formula(
                "GravY",
                false,
                "double GravY(double[] X) { return -26.042; } "
            );

            t_end = 2.7713;
            t_startUp = 0.0;

            dt = 4e-4;
            dt_startUp = 1e-5;

            //restartID = new Guid("");   // startUp session

            break;
        }
    case "Omega100": {

            rhoA = 0.33255;
            rhoB = 0.33255 / 1000;      
            sigma = 0.001;       // kg / s^2

            g = 26.042;         // cm / s^2
            GravityValue = new Formula(
                "GravY",
                false,
                "double GravY(double[] X) { return -26.042; } "
            );

            t_end = 27.713;
            t_startUp = 0.0;

            dt = 1e-3;
            dt_startUp = 1e-3;

            //restartID = new Guid("");   // startUp session

            break;
        }
} 

## Grid creation

In [ ]:
int[] Resolutions = new int[] { 4 };    // cells per radius R { 1, 2, 4, 8, 16 };
IGridInfo[] Grids = new IGridInfo[Resolutions.Length];

bool useSymmetry = true;

for(int i = 0; i < Resolutions.Length; i++) {
    int Res = Resolutions[i];

    string GridName;
    if(useSymmetry)
        GridName = $"CapillaryRise_{Res}x{8*Res}_halfTube";
    else
        GridName = $"CapillaryRise_{Res}x{8*Res}_fullTube";

    IGridInfo cachedGrid = BoSSSshell.WorkflowMgm.Grids.FirstOrDefault(grid => grid.Name == GridName);
    // cachedGrid = null;
    if(cachedGrid == null) {
        
        // must create new Grid
        double[] Xnodes;
        if(useSymmetry)
            Xnodes = GenericBlas.Linspace(0, R, Res + 1);
        else
            Xnodes = GenericBlas.Linspace(-R, R, (2 * Res) + 1);
        double[] Ynodes = GenericBlas.Linspace(0, H, 8 * Res + 1);
        var grd = Grid2D.Cartesian2DGrid(Xnodes, Ynodes);
    
        grd.EdgeTagNames.Add(1, "wall_lower");
        grd.EdgeTagNames.Add(2, "pressure_outlet_upper");

        if(useSymmetry)
            grd.EdgeTagNames.Add(3, "slipsymmetry_left");
        else
            grd.EdgeTagNames.Add(3, "navierslip_linear_left");

        grd.EdgeTagNames.Add(4, "navierslip_linear_right");

        grd.DefineEdgeTags(delegate (double[] X) {
            byte et = 0;
            if(Math.Abs(X[1]) <= 1.0e-8)
                et = 1;
            if(Math.Abs(X[1] - H) <= 1.0e-8)
                et = 2;
            if(useSymmetry) {
                if(Math.Abs(X[0]) <= 1.0e-8)
                    et = 3;
            } else {
                if(Math.Abs(X[0] + R) <= 1.0e-8)
                    et = 3;
            }
            if(Math.Abs(X[0] - R) <= 1.0e-8)
                et = 4;

            return et;
        });     
        
        grd.Name = GridName;
        
        Grids[i] = BoSSSshell.WorkflowMgm.SaveGrid(grd);
        //Grids[i] = grd;
        
    } else {
        //Console.WriteLine($"type: {cachedGrid.GetType()}, is IGridInfo? {cachedGrid is IGridInfo}");
        Console.WriteLine("Grid already found in database - identifid by name " + GridName);
        Grids[i] = cachedGrid;
    }
    
}

Opening existing database 'D:\local\CapillaryRise'.
Grid already found in database - identifid by name CapillaryRise_4x32_halfTube


## Control settings

In [ ]:
List<XNSE_Control> Controls = new List<XNSE_Control>();
Controls.Clear();

In [ ]:
for(int gIdx = 0; gIdx < Resolutions.Length; gIdx++) { 

var C = new XNSE_Control();

int k = 2;
C.SetDGdegree(k);


// physical parameters
// ===================
C.PhysicalParameters.rho_A = rhoA;
C.PhysicalParameters.mu_A = muA;
C.PhysicalParameters.rho_B = rhoB;
C.PhysicalParameters.mu_B = muB;

C.PhysicalParameters.Sigma = sigma;

C.PhysicalParameters.betaS_A = betaS_A;
C.PhysicalParameters.betaS_B = betaS_B;

C.PhysicalParameters.betaL = betaL;
C.PhysicalParameters.theta_e = theta_e;

C.PhysicalParameters.IncludeConvection = !startUp;
C.PhysicalParameters.Material = true;

    
// startUp?
// ========
if (startUp) {
    // set grid
    C.SetGrid(Grids[gIdx]);
    
    // initial conditions
    //double h0 = 1e-2;
    //Func<double[], double> PhiFunc = (X => X[1] - h0);
    Formula PhiFunc = new Formula(
        "Phi",
        false,
        "double Phi(double[] X) { return X[1] - 1e-2; } "
    );
    C.AddInitialValue("Phi", PhiFunc);

    //Func<double[], double> GravFunc = (X => -g);
    C.AddInitialValue("GravityY#A", GravityValue);
    C.AddInitialValue("GravityY#B", GravityValue);

} else {

    if(ts_restart > 0)
        C.RestartInfo = new Tuple<Guid, BoSSS.Foundation.IO.TimestepNumber>(restartID, ts_restart);
    else
        C.RestartInfo = new Tuple<Guid, BoSSS.Foundation.IO.TimestepNumber>(restartID, null);

}

// boundary conditions
// ===================
if(startUp) {
    C.AddBoundaryValue("wall_lower");
} else {
    C.AddBoundaryValue("wall_lower");
    C.ChangeBoundaryCondition("wall_lower", "pressure_outlet_lower");
    //C.AddBoundaryValue("pressure_outlet_lower");
}
C.AddBoundaryValue("pressure_outlet_upper");

if(useSymmetry)
    C.AddBoundaryValue("slipsymmetry_left");
else
    C.AddBoundaryValue("navierslip_linear_left");

C.AddBoundaryValue("navierslip_linear_right");

C.AdvancedDiscretizationOptions.GNBC_Localization = NavierSlip_Localization.Bulk;
C.AdvancedDiscretizationOptions.GNBC_SlipLength = NavierSlip_SlipLength.Prescribed_SlipLength;
C.PhysicalParameters.sliplength = Lslip;


// misc. solver options
// ====================
C.LSContiProjectionMethod = ContinuityProjectionOption.ConstrainedDG;

C.NonLinearSolver.SolverCode = NonLinearSolverCode.Picard;
C.NonLinearSolver.ConvergenceCriterion = 1e-8;
C.NonLinearSolver.MaxSolverIterations = 20;
C.LevelSet_ConvergenceCriterion = 1e-6;

C.AdvancedDiscretizationOptions.FilterConfiguration = CurvatureAlgorithms.FilterConfiguration.Default;
C.AdvancedDiscretizationOptions.SurfStressTensor = SurfaceSressTensor.Isotropic;
C.AdvancedDiscretizationOptions.SST_isotropicMode = SurfaceStressTensor_IsotropicMode.LaplaceBeltrami_ContactLine;


C.AdaptiveMeshRefinement = true;
int AMRlevel = 1;
C.activeAMRlevelIndicators.Add(new AMRonNarrowband() { maxRefinementLevel = AMRlevel });
C.activeAMRlevelIndicators.Add(new AMRLevelIndicatorLibrary.AMRonBoundary(new byte[] { 4 }) { maxRefinementLevel = AMRlevel });
if(!useSymmetry)
    C.activeAMRlevelIndicators.Add(new AMRLevelIndicatorLibrary.AMRonBoundary(new byte[] { 3 }) { maxRefinementLevel = AMRlevel });
C.AMR_startUpSweeps = AMRlevel;


// level-set
// =========
C.Option_LevelSetEvolution = LevelSetEvolution.FastMarching;


// Timestepping
// ============
C.TimeSteppingScheme = TimeSteppingScheme.ImplicitEuler;
C.Timestepper_LevelSetHandling = LevelSetHandling.Coupled_Once;

C.TimesteppingMode = AppControl._TimesteppingMode.Transient;
C.dtFixed = (startUp) ? dt_startUp : dt;
C.Endtime = (startUp) ? Math.Max(Math.Sqrt(2 * R / g), 2 * R * muA / sigma) * 4.0 : (t_startUp + t_end);
C.NoOfTimesteps = (int)(C.Endtime / C.dtMin);
C.saveperiod = 100;

        
    
C.PostprocessingModules.Add(new BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases.MovingContactLineLogging() { LogPeriod = 100 });
    
double Res = Resolutions[gIdx];
if (startUp) 
    C.SessionName = $"CapillaryRise_{omegaTc}_{Res}x{8*Res}AMR{AMRlevel}_startUp";
else
    C.SessionName = $"CapillaryRise_{omegaTc}_{Res}x{8*Res}AMR{AMRlevel}";
    
    
Controls.Add(C);

}

## Launch job

In [ ]:
Controls.Select(C => C.SessionName)

index,value
0,CapillaryRise_Omega1_4x32AMR1_startUp


In [ ]:
foreach(var ctrl in Controls) {
    var oneJob              = ctrl.CreateJob();
    oneJob.NumberOfMPIProcs = 1;
    oneJob.Activate(myBatch); 
}

Deploying job CapillaryRise_Omega1_4x32AMR1_startUp ... 
Deploying executables and additional files ...
Deployment directory: \\hpccluster\hpccluster-scratch\smuda\binaries\CapillaryRise-XNSE_Solver2024Feb13_104158.036680
copied 42 files.
   written file: control.obj
   copied 'win\amd64' runtime.
deployment finished.



In [ ]:
// var testJob              = Controls.Pick(0).CreateJob();
// testJob.NumberOfMPIProcs = 1;
// testJob.Activate(myBatch); 

Deploying job CapillaryRise_Omega=1_4x32AMR1_startUp ... 
Deploying executables and additional files ...
Deployment directory: \\hpccluster\hpccluster-scratch\smuda\binaries\CapillaryRise-XNSE_Solver2024Feb09_092723.705523
copied 42 files.
   written file: control.obj
   copied 'win\amd64' runtime.
deployment finished.


Error: System.IO.IOException: Job is assigned to batch processor, but no deployment directory can be found.
   at BoSSS.Application.BoSSSpad.MetaJobMgrIO.RetryIOop(Func`2 op, String Message, Boolean SurpressException) in D:\BoSSS-experimental\public\src\L4-application\BoSSSpad\BatchProcessorClient.cs:line 454
   at BoSSS.Application.BoSSSpad.Job.<DeployExecuteables>g__TestWR|95_0() in D:\BoSSS-experimental\public\src\L4-application\BoSSSpad\Job.cs:line 1770
   at BoSSS.Application.BoSSSpad.Job.DeployExecuteables() in D:\BoSSS-experimental\public\src\L4-application\BoSSSpad\Job.cs:line 1868
   at BoSSS.Application.BoSSSpad.Job.Reactivate() in D:\BoSSS-experimental\public\src\L4-application\BoSSSpad\Job.cs:line 1487
   at BoSSS.Application.BoSSSpad.Job.Activate(BatchProcessorClient bpc) in D:\BoSSS-experimental\public\src\L4-application\BoSSSpad\Job.cs:line 1446
   at Submission#13.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.Cod

In [ ]:
// testJob.LatestDeployment

<null>

In [ ]:
BoSSSshell.WorkflowMgm.Sessions